In [269]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import subprocess

SCAP = 94140000 # salary cap
LTL = 113290000 # luxury tax limit
CHEAP = 5000000
CHEAP2 = 20000000
CHEAP3 = 65000000
EXP = 200000000
EXP2 = 500000000
EXP3 = 1000000000
MP = 15 # max players
Z=2 # min position players

In [201]:
def get_pid(player, df):
    return df[df['Player'] == player].index.tolist()[0]

def get_cid(column, df):
    return np.where(df.columns == column)[0] + 1

def get_salary(player, df):
    return int(df[df['Player'] == player]['2016-17'].tolist()[0][1:])

In [202]:
df = pd.read_csv('bball_data.csv')
salaries = pd.read_csv('bball_salaries.csv')

In [268]:
single = lambda x: x.unique()[0]
wavg = lambda x: (x * df.ix[x.index]['G']).sum()/df.ix[x.index]['G'].sum()
cleaned = df.groupby(['Rk'], as_index=False).agg(
    {'Player': single,
     'Pos': single,
     'Age': single,
     'G': np.sum,
     'GS': np.sum,
     'MP': wavg,
     'FG': wavg,
     'FGA': wavg,
     'FG%': wavg,
     '3P': wavg,
     '3PA': wavg,
     '3P%': wavg,
     '2P': wavg,
     '2PA': wavg,
     '2P%': wavg,
     'eFG%': wavg,
     'FT': wavg,
     'FTA': wavg,
     'FT%': wavg,
     'ORB': wavg,
     'DRB': wavg,
     'TRB': wavg,
     'AST': wavg,
     'STL': wavg,
     'BLK': wavg,
     'TOV': wavg,
     'PF': wavg,
     'PS/G': wavg})

In [270]:
pos = cleaned['Pos'].unique()

In [271]:
cleaned.dropna(inplace=True) # removing players from consideration that have NaN for a certain stat
textdata = cleaned[['Pos', 'Player']].copy()
cleaned.drop(['Rk', 'G', 'GS', 'MP', 'Age', 'Pos', 'Player'],
             axis=1, inplace=True)
scaler = preprocessing.MinMaxScaler()
cleaned = pd.DataFrame(scaler.fit_transform(cleaned),
                       columns = cleaned.columns, index=cleaned.index)

In [272]:
cleaned = cleaned.join(textdata)
cleaned.rename(columns={'FT%':'FTP', 'eFG%': 'eFGP',
                        '2P%': 'TWPP', 'THP%': 'THPP',
                        'FG%':'FGP', 'THP': 'THP',
                        '2PA': 'TWPA', '3PA':'THPA', 
                        '2P': 'TWP', 'PS/G': 'PSG'}, inplace=True)
# cleaned.columns

In [267]:
coeffs = {
    'AST': 2.5,
    'TRB': 2.25,
    'PSG': 2,
    'THP': 1.5,
    'STL': 3,
    'BLK': 3,
    'TOV': -1.5,
    'PF': -1,
}
plims = {
    '0' : '',
    '1' : '',
    '2' : '',
    '3' : '',
    '4' : '',
}
of = 'max: '
c1 = ''
c2 = ''
vrs = 'bin '
for r in cleaned.itertuples():
    p = str(np.where(pos == r[get_cid('Pos', cleaned)])[0][0])
    R = 'R' + str(r[0]) + p
    v = 0
    for i in cleaned.columns:
        if get_cid(i, cleaned) < 23:
            c = 1
            if i in coeffs:
                c = coeffs[i]
            v += c * r[get_cid(i, cleaned)]
    of += ' ' + str(v) + ' ' + R + ' +'
    c1 += str(get_salary(r[24], salaries)) + ' ' + R + ' + '
    c2 += R + ' + '
    vrs += R + ', '
    plims[p] += R + ' + '
            
#     print(get_salary(r[get_cid('Player', cleaned)], salaries))
of = of[:-2] + ';'
c1 = c1[:-2] + '<=' + str(EXP3) + ';' #CHANGE HERE FOR BUDGET DIFFERENCES
c2 = c2[:-2] + '<=' + str(MP) + ';'
vrs = vrs[:-2] + ';'
# print(of)

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:25: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:33: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


In [266]:
for k, v in plims.items():
    plims[k] = v[:-2] + '>=' + str(Z) + ';'
#     print(plims[k])

In [265]:
# print(c1)

In [264]:
# print(c2)

In [263]:
# print(vrs)

In [273]:
# simple file logic

# try:
#     with open('simpleLP2_X.txt', 'wx') as f: # change 'X' to iteration identifier
#         f.write(of + '\n')
#         f.write(c1 + '\n')
#         f.write(c2 + '\n')
#         f.write(vrs + '\n')
#     subprocess.run('./lpsolve simpleLP2_X.txt > simpleLP2_X_a.txt') # not tested, but you get the idea...
# except FileExistsError:
#     print('That file already exists! Pick a new name...')
#     print('Failed to write file!')

In [259]:
# The results of some different LPs...
players11 = [15, 64, 95, 247, 273, 327, 342, 397, 404, 427]
players12 = [15, 64, 147, 247, 273, 327, 342, 397, 404, 427]
players21 = [15, 86, 87, 93, 95, 118, 147, 220, 273, 327, 342, 397, 404, 407, 424] # scap
players22 = [15, 86, 87, 93, 95, 118, 147, 273, 327, 340, 342, 397, 404, 407, 422] #ltl
players23 = [108, 129, 206, 239, 279, 284, 296, 302, 306, 346, 399, 428, 438, 439] #cheap
players24 = [15, 55, 87, 108, 147, 158, 220, 224, 232, 273, 303, 346, 351, 407, 439] #cheap2
players25 = [15, 51, 86, 87, 106, 118, 147, 220, 273, 340, 342, 370, 397, 404, 407] #cheap3
players26 = [15, 64, 86, 95, 114, 118, 147, 164, 247, 273, 327, 342, 404, 422, 427] #EXP
players27 = [15, 64, 86, 93, 95, 103, 114, 164, 208, 247, 258, 289, 404, 422, 427] #EXP2
players28 = [15, 64, 86, 93, 95, 103, 114, 164, 208, 247, 258, 289, 404, 422, 427] #EXP3 Same as above!
total_salary = 0
for pl in players27:
    pls = cleaned.loc[pl]['Player']
    print(pls)
    total_salary += int(get_salary(pls, salaries))
print(total_salary)

Giannis Antetokounmpo\antetgi01
Jimmy Butler\butleji01
DeMarcus Cousins\couside01
Stephen Curry\curryst01
Anthony Davis\davisan02
DeMar DeRozan\derozde01
Kevin Durant\duranke01
James Harden\hardeja01
LeBron James\jamesle01
Kawhi Leonard\leonaka01
Kevin Love\loveke01
Paul Millsap\millspa01
Karl-Anthony Towns\townska01
John Wall\walljo01
Russell Westbrook\westbru01
290652320


In [240]:
cost1 = [92305992, 112301454]
value1 = [142.00936028, 146.84077117]
cost2 = [4926626, 19937527, 64998342, 93555745, 112920206, 199657770, 290652320]
value2 = [74.57695501, 146.15246622, 188.37375013, 199.65147302, 205.53770810, 224.22243852, 229.30263647]

In [ ]:
# simplelp1_LTL
# Value of objective function: 146.84077117

# Actual values of the variables:

# R150                            1

# R643                            1

# R952                            1

# R2473                           1

# R2730                           1

# R3271                           1

# R3421                           1

# R3974                           1

# R4042                           1

# R4274                           1

# simplelp1_SCAP
# Value of objective function: 142.00936028

# Actual values of the variables:

# R150                            1

# R643                            1

# R1472                           1

# R2473                           1

# R2730                           1

# R3271                           1

# R3421                           1

# R3974                           1

# R4042                           1

# R4274                           1